<a href="https://colab.research.google.com/github/joako-m-g/2048IA/blob/main/src/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set Up

In [20]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from funcs_aux import rolling_sales


In [7]:
data = pd.read_csv("drive/MyDrive/datos_unidos.csv")

In [9]:
print(data.columns)

Index(['SKU', 'DATE', 'STORE_ID', 'PRICE', 'QUANTITY', 'TOTAL_SALES', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'OPENDATE', 'CLOSEDATE',
       'STORE_SUBGROUP_DATE_ID', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'INITIAL_TICKET_PRICE', 'BASE_PRICE',
       'COSTOS', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE',
       'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK'],
      dtype='object')


In [10]:
print(data.head())

          SKU        DATE STORE_ID  PRICE  QUANTITY  TOTAL_SALES     REGION  \
0  BEAHASH001  2021-01-01   S00068  35.53        10       355.30       West   
1  BEAHASH001  2021-01-01   S00086  33.52         2        67.04  Southeast   
2  BEAHASH001  2021-01-01   S00124  37.61         2        75.22  Northeast   
3  BEAHASH001  2021-01-01   S00140  34.51         4       138.04  Southeast   
4  BEAHASH001  2021-01-02   S00013  33.77         1        33.77  Southwest   

         CITY STATE STORE_TYPE  ... COSTOS YEAR_OPEN YEAR_CLOSE MONTH_OPEN  \
0    Lakewood    CO    Express  ...  15.07      2018       2025         11   
1     Raleigh    NC    Express  ...  15.07      2011       2025         12   
2        Erie    PA     Outlet  ...  15.07      2018       2025          5   
3  Greenville    SC       Mall  ...  15.07      2011       2025          5   
4     El Paso    TX       Mall  ...  15.07      2019       2025         10   

  MONTH_CLOSE  YEAR MONTH DAY DAY_OF_WEEK  WEEK  
0     

In [11]:
print(len(data))

6825953


## Rolling Features

Incluimos medias moviles y desviaciones estandar para captar dinamica temporal. Estas las calculamos teniendo en cuenta los dias de cero ventas. Debido a recursos computacionales no podemos expandir todo el dataset.

In [22]:
# Medias/desviaciones móviles por SKU
data = rolling_sales(data, group='SKU', windows=[7, 30, 90], std=True)

# Medias/desviaciones móviles por SUBGROUP
data = rolling_sales(data, group='SUBGROUP', windows=[7, 30, 90], std=True)

# Medias/desviaciones móviles por STORE_ID
data = rolling_sales(data, group='STORE_ID', windows=[7, 30, 90], std=True)

KeyError: "['S', 'K', 'U', '_', 's', 't', 'd', '7', 'D'] not in index"

In [ ]:
print(data.columns)

In [ ]:
print(data.memory_usage) # Verificamos cuanta memoria ocupa ahora

In [ ]:
data.to_csv("../data/procesados/datos_rolling.csv")

## Linear Regression

Entrenamos con el primer 10% a ver como se comporta

In [ ]:
df_10pct_series = data.groupby(['SKU', 'Store_ID'], group_keys=False).apply(lambda x: x.head(int(len(x) * 0.1)))

In [ ]:
X = df_10pct_series[categorical_cols + numeric_cols]  # variables predictoras
y = df_10pct_series['TOTAL_SALES']                    # variable objetivo

In [ ]:
# OneHotEncoder para variables categóricas (transforma texto en variables binarias)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Passthrough para variables numéricas (se dejan igual)
numeric_transformer = 'passthrough'

# ColumnTransformer aplica transformaciones diferentes a columnas distintas
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, categorical_cols),
        ('numeric', numeric_transformer, numeric_cols)
    ]
)

In [ ]:
# Definimos el Pipeline
# Combina el preprocesador y el modelo en un solo flujo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

NameError: name 'Pipeline' is not defined

In [ ]:
# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

NameError: name 'train_test_split' is not defined

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Métricas
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

puta